In [6]:
import pandas as pd

# Read the data from the CSV file
df = pd.read_csv('Begrotingsstaten_2013-2025.csv')

# Convert the relevant columns to appropriate data types
df['Realisatie'] = pd.to_numeric(df['Realisatie'], errors='coerce')

# Aggregate the sums for each chapter
chapter_expenses = df.groupby(['HoofdstukRomeins', 'Hoofdstuknaam'])[['Realisatie']].sum().reset_index()

# Display the results
print(chapter_expenses)

# If you want to save the result to a new CSV file
chapter_expenses.to_csv('chapter_expenses_overview.csv', index=False)


   HoofdstukRomeins                                      Hoofdstuknaam  \
0                 A                                Infrastructuurfonds   
1                 A                                   Mobiliteitsfonds   
2                 B                                      Gemeentefonds   
3                 C                                     Provinciefonds   
4                 F                               Diergezondheidsfonds   
5                 H                                          BES-fonds   
6                 I                                          De Koning   
7               IIA                                    Staten-Generaal   
8               IIB      Overige Hoge Colleges van Staat en Kabinetten   
9               IIB  Overige Hoge Colleges van Staat, Kabinetten en...   
10              IIB  Overige Hoge Colleges van Staat, Kabinetten en...   
11              III                                     Algemene Zaken   
12               IV                   

In [24]:
import pandas as pd

# Read the data from the CSV file
df = pd.read_csv('Begrotingsstaten_2013-2025.csv')

# Convert the relevant columns to appropriate data types
df['Realisatie'] = pd.to_numeric(df['Realisatie'], errors='coerce')

# if first 7 characters of Hoofdstuknaam are the same, combine them
# Create a dictionary to store the last used name for each prefix
name_mapping = {}

def combine_names(name):
    prefix = name[:7]
    if prefix in name_mapping:
        return name_mapping[prefix]
    else:
        name_mapping[prefix] = name
        return name

df['Hoofdstuknaam'] = df['Hoofdstuknaam'].apply(combine_names)

# Aggregate the sums for each chapter
chapter_expenses = df.groupby(['Begrotingsjaar', 'Hoofdstuknaam'])[['Realisatie']].sum().reset_index()

# Rename columns for better readability
chapter_expenses.columns = ['Begrotingsjaar', 'Afdeling', 'Bedrag']

# Display the results
for index, row in chapter_expenses.iterrows():
    print(f"{row['Afdeling']: <40} {row['Bedrag']}")

# If you want to save the result to a new CSV file
chapter_expenses.to_csv('agency_expenses_overview.csv', index=False)


Algemene Zaken                           127113
BES-fonds                                71376
Binnenlandse Zaken en Koninkrijksrelaties 1756405
Buitenlandse Zaken                       22407584
De Koning                                81630
Defensie                                 15560334
Deltafonds                               3659780
Diergezondheidsfonds                     61623
Economische Zaken                        30360006
Financien                                -4682622
Gemeentefonds                            35984515
Infrastructuurfonds                      38324263
Koninkrijksrelaties                      466239
Nationale Schuld (Transactiebasis)       153001108
Onderwijs, Cultuur en Wetenschap         36408722
Overige Hoge Colleges van Staat en Kabinetten 239116
Provinciefonds                           3105708
Sociale Zaken en Werkgelegenheid         61527186
Staten-Generaal                          284275
Veiligheid en Justitie                   20776399
Volksgezondhe

In [26]:
import pandas as pd

# Read the data from the CSV file
df = pd.read_csv('Begrotingsstaten_2013-2025.csv')

# Ensure 'Bedrag' is numeric
df['Bedrag'] = pd.to_numeric(df['Realisatie'], errors='coerce')

# if first 7 characters of Hoofdstuknaam are the same, combine them
# Create a dictionary to store the last used name for each prefix
name_mapping = {}

def combine_names(name):
    prefix = name[:7]
    if prefix in name_mapping:
        return name_mapping[prefix]
    else:
        name_mapping[prefix] = name
        return name
    
def other_combine(name):
    if name == 'Veiligheid en Justitie':
        return 'Justitie en Veiligheid'
    if name == 'Wonen & Rijksdienst':
        return 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer'
    return name

df['Hoofdstuknaam'] = df['Hoofdstuknaam'].apply(combine_names)
df['Hoofdstuknaam'] = df['Hoofdstuknaam'].apply(other_combine)

df['Afdeling'] = df['Hoofdstuknaam']

# Aggregate the sums for each combination of 'Hoofdstuknaam' and 'Begrotingsjaar'
df_aggregated = df.groupby(['Afdeling', 'Begrotingsjaar'])['Bedrag'].sum().reset_index()

# Pivot the data so that each year becomes a column
pivot_df = df_aggregated.pivot(index='Afdeling', columns='Begrotingsjaar', values='Bedrag').reset_index()

# Render the DataFrame to an HTML table
def render_html_table(pivot_df):
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Spend | DODGE: Department of Dutch Government Efficiency</title>
        <link rel="icon" href="DODGE.png">
        <link rel="stylesheet" href="style.css">
        <!-- Include Tailwind CSS -->
        <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
    </head>
    <body>
        <div class="country-notice">
        <p>🇳🇱 An official website of Merijn Vervoorn</p>
    </div>
    <div class="navbar-container">
        <div class="navbar">
            <div class="title flex items-center" >
                <a href="index.html">
                    <img src="DODGE.png" alt="Logo" loading="lazy">
                </a>
                <div>
                    <a class="h1" href="index.html">Department of Dutch Government Efficiency</a><br>
                    <a class="h2" href="index.html">The people voted for major reform.</a>
                </div>
            </div>
            <div class="menu flex gap-4">
                <a href="spend.html" class="active">Spend</a>
                <a href="about.html" class="hover:text-gray-200">About</a>
            </div>
        </div>
    </div>
    <div class="line"></div>
    <div class="container mx-auto p-4">
        <h1 class="title-spend">Agency Spending</h1>
        <table>
            <thead>
                <tr>
                    <th>Agency</th>
    """
    for year in pivot_df.columns[1:]:
        html_content += f"<th>{year}</th>"
    html_content += """
                </tr>
            </thead>
            <tbody>
    """
    for _, row in pivot_df.iterrows():
        html_content += "<tr>"
        html_content += f"<td>{row['Afdeling']}</td>"
        for year in pivot_df.columns[1:]:
            amount = row[year]
            if pd.isna(amount):
                html_content += "<td>N/A</td>"
            else:
                formatted_amount = f"€{amount:,.0f}"
                html_content += f"<td>{formatted_amount}</td>"
        html_content += "</tr>"

    html_content += """
            </tbody>
        </table>
    </div>
    </body>
    </html>
    """
    return html_content

# Generate the HTML content
html_content = render_html_table(pivot_df)

# Write the HTML content to a file
with open('../spend-full.html', 'w') as file:
    file.write(html_content)

print("HTML file has been generated: spend-full.html")


HTML file has been generated: spend-full.html


In [27]:
import pandas as pd

# Read the data from the CSV file
df = pd.read_csv('Begrotingsstaten_2013-2025.csv')

# Ensure 'Bedrag' is numeric
df['Bedrag'] = pd.to_numeric(df['StandOWB'], errors='coerce')

# if first 7 characters of Hoofdstuknaam are the same, combine them
# Create a dictionary to store the last used name for each prefix
name_mapping = {}

def combine_names(name):
    prefix = name[:7]
    if prefix in name_mapping:
        return name_mapping[prefix]
    else:
        name_mapping[prefix] = name
        return name
    
def other_combine(name):
    if name == 'Veiligheid en Justitie':
        return 'Justitie en Veiligheid'
    if name == 'Wonen & Rijksdienst':
        return 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer'
    return name

df['Hoofdstuknaam'] = df['Hoofdstuknaam'].apply(combine_names)
df['Hoofdstuknaam'] = df['Hoofdstuknaam'].apply(other_combine)

df['Afdeling'] = df['Hoofdstuknaam']

# Aggregate the sums for each combination of 'Hoofdstuknaam' and 'Begrotingsjaar'
df_aggregated = df.groupby(['Afdeling', 'Begrotingsjaar'])['Bedrag'].sum().reset_index()

# Pivot the data so that each year becomes a column
pivot_df = df_aggregated.pivot(index='Afdeling', columns='Begrotingsjaar', values='Bedrag').reset_index()

pivot_2019_2025 = pivot_df[['Afdeling', 2019, 2025]]

# Render the DataFrame to an HTML table
def render_html_table(pivot_df):
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Spend | DODGE: Department of Dutch Government Efficiency</title>
        <link rel="icon" href="DODGE.png">
        <link rel="stylesheet" href="style.css">
        <!-- Include Tailwind CSS -->
        <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
    </head>
    <body>
        <div class="country-notice">
        <p>🇳🇱 An official website of Merijn Vervoorn</p>
    </div>
    <div class="navbar-container">
        <div class="navbar">
            <div class="title flex items-center" >
                <a href="index.html">
                    <img src="DODGE.png" alt="Logo" loading="lazy">
                </a>
                <div>
                    <a class="h1" href="index.html">Department of Dutch Government Efficiency</a><br>
                    <a class="h2" href="index.html">The people voted for major reform.</a>
                </div>
            </div>
            <div class="menu flex gap-4">
                <a href="spend.html" class="active">Spend</a>
                <a href="about.html" class="hover:text-gray-200">About</a>
            </div>
        </div>
    </div>
    <div class="line"></div>
    <div class="container mx-auto p-4">
        <h1 class="title-spend">Agency Spending</h1>
        <table>
            <thead>
                <tr>
                    <th>Agency</th>
    """
    for year in pivot_2019_2025.columns[1:]:
        html_content += f"<th>{year}</th>"
    html_content += """
                </tr>
            </thead>
            <tbody>
    """
    for _, row in pivot_2019_2025.iterrows():
        html_content += "<tr>"
        html_content += f"<td>{row['Afdeling']}</td>"
        for year in pivot_2019_2025.columns[1:]:
            amount = row[year]
            if pd.isna(amount):
                html_content += "<td>N/A</td>"
            else:
                formatted_amount = f"€{amount:,.0f}"
                html_content += f"<td>{formatted_amount}</td>"
        html_content += "</tr>"

    html_content += """
            </tbody>
        </table>
    </div>
    </body>
    </html>
    """
    return html_content

# Generate the HTML content
html_content = render_html_table(pivot_df)

# Write the HTML content to a file
with open('../spend.html', 'w') as file:
    file.write(html_content)

print("HTML file has been generated: spend.html")


HTML file has been generated: spend.html
